Attempting to create a function that will take in a dataframe and push back out the appropriate url, as when looking at mttggoldfish.com's url structure, cards are identified by set name and then card name, among some other discerning factors.

In [35]:
import pandas as pd
from rapidfuzz import fuzz
import re

In [36]:

#Reading in test card set
mtg_cards_df = pd.read_csv('MTG_Cards.csv')
mtg_cards_df

,QUANTITY,NM_VALUE,SET_CODE,SET_NAME,NAME,RARITY,COLLECTOR_NUMBER.,FOIL,NOTES,SCRYFALL_ID
0,1,5.35,2X2,Double Masters 2022,Darksteel Plate,Rare,304,NaN,NaN,3f99fb1e-99a6-4c83-98eb-7bff23996a7f
1,1,36.56,2X2,Double Masters 2022,Mana Drain,Mythic,57,NaN,NaN,3c429c40-2389-41e5-8681-4bb274e25eba
2,1,32.83,2X2,Double Masters 2022,Mana Drain,Mythic,57,FOIL,NaN,3c429c40-2389-41e5-8681-4bb274e25eba
3,1,1.86,AFC,Forgotten Realms Commander,Gratuitous Violence,Rare,127,NaN,NaN,73a5760c-e412-4bb4-b176-cfd0e8fc8f95
4,1,2.15,AFC,Forgotten Realms Commander,Hurl Through Hell,Rare,48,NaN,NaN,47a3321c-28ed-4cd7-84d8-c8cecc09348b
...,...,...,...,...,...,...,...,...,...,...
68,1,2.93,WOC,Wilds of Eldraine Commander,Court of Ardenvale,Rare,29,NaN,NaN,af7f7659-1f20-45b6-bcd2-21879beb7849
69,1,47.53,WOT,Wilds of Eldraine: Enchanting Tales,Smothering Tithe,Mythic,67,NaN,NaN,58178dd4-c61e-4017-b2e6-2d8308ded4fd
70,2,3.21,ZNR,Zendikar Rising,Angel of Destiny,Mythic,2,NaN,NaN,9897074a-0ac4-4d1a-9aac-e77830cc5c78
71,2,4.80,ZNR,Zendikar Rising,Valakut Awakening // Valakut Stoneforge,Rare,174,NaN,NaN,228e551e-023a-4c9a-8f32-58dae6ffdf7f


In [37]:
#Dropping irrelevent columns
mtg_cards_df.drop(columns=['NM_VALUE', 'SET_CODE', 'SCRYFALL_ID', 'NOTES'], inplace=True )
mtg_cards_df

,QUANTITY,SET_NAME,NAME,RARITY,COLLECTOR_NUMBER.,FOIL
0,1,Double Masters 2022,Darksteel Plate,Rare,304,NaN
1,1,Double Masters 2022,Mana Drain,Mythic,57,NaN
2,1,Double Masters 2022,Mana Drain,Mythic,57,FOIL
3,1,Forgotten Realms Commander,Gratuitous Violence,Rare,127,NaN
4,1,Forgotten Realms Commander,Hurl Through Hell,Rare,48,NaN
...,...,...,...,...,...,...
68,1,Wilds of Eldraine Commander,Court of Ardenvale,Rare,29,NaN
69,1,Wilds of Eldraine: Enchanting Tales,Smothering Tithe,Mythic,67,NaN
70,2,Zendikar Rising,Angel of Destiny,Mythic,2,NaN
71,2,Zendikar Rising,Valakut Awakening // Valakut Stoneforge,Rare,174,NaN


In [38]:
mtg_cards_df.describe()

,QUANTITY
count,73.000000
mean,1.191781
std,0.569052
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,4.000000


In [39]:
mtg_cards_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   QUANTITY           73 non-null     int64 
 1   SET_NAME           73 non-null     object
 2   NAME               73 non-null     object
 3   RARITY             73 non-null     object
 4   COLLECTOR_NUMBER.  73 non-null     object
 5   FOIL               13 non-null     object
dtypes: int64(1), object(5)
memory usage: 3.5+ KB


In [40]:
# Stripping whitespace from column names
mtg_cards_df.columns = mtg_cards_df.columns.str.replace(' ', '')
mtg_cards_df

,QUANTITY,SET_NAME,NAME,RARITY,COLLECTOR_NUMBER.,FOIL
0,1,Double Masters 2022,Darksteel Plate,Rare,304,NaN
1,1,Double Masters 2022,Mana Drain,Mythic,57,NaN
2,1,Double Masters 2022,Mana Drain,Mythic,57,FOIL
3,1,Forgotten Realms Commander,Gratuitous Violence,Rare,127,NaN
4,1,Forgotten Realms Commander,Hurl Through Hell,Rare,48,NaN
...,...,...,...,...,...,...
68,1,Wilds of Eldraine Commander,Court of Ardenvale,Rare,29,NaN
69,1,Wilds of Eldraine: Enchanting Tales,Smothering Tithe,Mythic,67,NaN
70,2,Zendikar Rising,Angel of Destiny,Mythic,2,NaN
71,2,Zendikar Rising,Valakut Awakening // Valakut Stoneforge,Rare,174,NaN


In [41]:
#Changing column names to Capitalized
columns = mtg_cards_df.columns
mtg_cards_df.columns = mtg_cards_df.columns.str.capitalize()

In [42]:
#Changing foils to 0 and 1, contemplating creating a predictive model
#after this project is complete
mtg_cards_df.replace('FOIL', 1, inplace=True)
mtg_cards_df['Foil'].fillna(0, inplace=True)
mtg_cards_df

/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_11972/2309731801.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mtg_cards_df.replace('FOIL', 1, inplace=True)
/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_11972/2309731801.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.




,Quantity,Set_name,Name,Rarity,Collector_number.,Foil
0,1,Double Masters 2022,Darksteel Plate,Rare,304,0.0
1,1,Double Masters 2022,Mana Drain,Mythic,57,0.0
2,1,Double Masters 2022,Mana Drain,Mythic,57,1.0
3,1,Forgotten Realms Commander,Gratuitous Violence,Rare,127,0.0
4,1,Forgotten Realms Commander,Hurl Through Hell,Rare,48,0.0
...,...,...,...,...,...,...
68,1,Wilds of Eldraine Commander,Court of Ardenvale,Rare,29,0.0
69,1,Wilds of Eldraine: Enchanting Tales,Smothering Tithe,Mythic,67,0.0
70,2,Zendikar Rising,Angel of Destiny,Mythic,2,0.0
71,2,Zendikar Rising,Valakut Awakening // Valakut Stoneforge,Rare,174,0.0


In [43]:
mtg_cards_df['Set_name'].values

array(['Double Masters 2022', 'Double Masters 2022',
       'Double Masters 2022', 'Forgotten Realms Commander',
       'Forgotten Realms Commander', 'Forgotten Realms Commander',
       'Forgotten Realms Commander', 'Forgotten Realms Commander',
       'Forgotten Realms Commander', 'Forgotten Realms Commander',
       'Adventures in the Forgotten Realms',
       'Adventures in the Forgotten Realms',
       'Adventures in the Forgotten Realms',
       'Adventures in the Forgotten Realms', 'Battlebond', 'Battlebond',
       'Commander 2021', 'Commander 2021', 'Commander 2021',
       'Commander 2021', 'Commander 2021', 'Commander 2021',
       'Commander 2021', 'Commander 2021',
       "Commander Legends: Battle for Baldur's Gate",
       "Commander Legends: Battle for Baldur's Gate", 'Commander Legends',
       'Commander Legends', 'Commander Legends', 'Commander Legends',
       'Commander Legends', 'Duel Decks: Knights vs. Dragons',
       'Throne of Eldraine', 'Throne of Eldraine', 

I'm realizing here that the Set names in my list are not quite matching up with the set names as displayed on mtggoldfish.com's website. Additionally, I need to create columns denoting other features of a card, such as whether its prerelease, promo pack, bordless, anime, foil-etched, and all other factors that could be added to create the url.

In [44]:
mtg_cards_df['Set_name'] = mtg_cards_df['Set_name'].replace(' ', '+', regex=True)
mtg_cards_df

,Quantity,Set_name,Name,Rarity,Collector_number.,Foil
0,1,Double+Masters+2022,Darksteel Plate,Rare,304,0.0
1,1,Double+Masters+2022,Mana Drain,Mythic,57,0.0
2,1,Double+Masters+2022,Mana Drain,Mythic,57,1.0
3,1,Forgotten+Realms+Commander,Gratuitous Violence,Rare,127,0.0
4,1,Forgotten+Realms+Commander,Hurl Through Hell,Rare,48,0.0
...,...,...,...,...,...,...
68,1,Wilds+of+Eldraine+Commander,Court of Ardenvale,Rare,29,0.0
69,1,Wilds+of+Eldraine:+Enchanting+Tales,Smothering Tithe,Mythic,67,0.0
70,2,Zendikar+Rising,Angel of Destiny,Mythic,2,0.0
71,2,Zendikar+Rising,Valakut Awakening // Valakut Stoneforge,Rare,174,0.0


In [45]:
set_links_df = pd.read_csv('./card_spider/links.csv')
set_links_df

,set_link
0,/sets
1,/sets/65-10th-edition
2,/sets/1274-30th-anniversary-edition
3,/sets/1371-30th-anniversary-promos
4,/sets/68-7th-edition
...,...
366,/sets/10-zendikar
367,/sets/253-zendikar-expedition
368,/sets/339-zendikar-rising
369,/sets/364-zendikar-rising-expeditions


In [46]:
set_links_df = set_links_df[1:]
set_links_df

,set_link
1,/sets/65-10th-edition
2,/sets/1274-30th-anniversary-edition
3,/sets/1371-30th-anniversary-promos
4,/sets/68-7th-edition
5,/sets/67-8th-edition
...,...
366,/sets/10-zendikar
367,/sets/253-zendikar-expedition
368,/sets/339-zendikar-rising
369,/sets/364-zendikar-rising-expeditions


In [47]:
def extract_set_name(link):
    match = re.search(r'/sets/\d+-(.*)', link)
    if match:
        return match.group(1)
    return None

In [48]:
set_links_df['set_name'] = set_links_df['set_link'].apply(extract_set_name)
set_links_df

/var/folders/g_/5gwjpx2j5c7cg2mn_rrdzqpc0000gn/T/ipykernel_11972/4163503164.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set_links_df['set_name'] = set_links_df['set_link'].apply(extract_set_name)


,set_link,set_name
1,/sets/65-10th-edition,10th-edition
2,/sets/1274-30th-anniversary-edition,30th-anniversary-edition
3,/sets/1371-30th-anniversary-promos,30th-anniversary-promos
4,/sets/68-7th-edition,7th-edition
5,/sets/67-8th-edition,8th-edition
...,...,...
366,/sets/10-zendikar,zendikar
367,/sets/253-zendikar-expedition,zendikar-expedition
368,/sets/339-zendikar-rising,zendikar-rising
369,/sets/364-zendikar-rising-expeditions,zendikar-rising-expeditions


In [59]:
similarities = []

for set_name in mtg_cards_df['Set_name']:
    max_similarity = 0
    
    for set_name_stocks in set_links_df['set_name']:
        similarity = fuzz.ratio(set_name, set_name_stocks)
        if similarity > max_similarity:
            max_similarity = similarity
            best_set_name_stocks = set_name_stocks
        
    similarities.append((set_name, best_set_name_stocks, max_similarity))

Not perfect, some issues here, forgotten realms commander, core set 2020 promos, wilds of eldraine commander all matched with
incorrect name. Have to manually fix these.

In [61]:
similarity_df = pd.DataFrame(similarities, columns=['cards_set_name', 'stocks_set_name', 'Highest Similarity'])
similarity_df.head(40)

,cards_set_name,stocks_set_name,Highest Similarity
0,Double+Masters+2022,double-masters-2022,78.947368
1,Double+Masters+2022,double-masters-2022,78.947368
2,Double+Masters+2022,double-masters-2022,78.947368
3,Forgotten+Realms+Commander,art-series-commander-masters,48.148148
4,Forgotten+Realms+Commander,art-series-commander-masters,48.148148
5,Forgotten+Realms+Commander,art-series-commander-masters,48.148148
6,Forgotten+Realms+Commander,art-series-commander-masters,48.148148
7,Forgotten+Realms+Commander,art-series-commander-masters,48.148148
8,Forgotten+Realms+Commander,art-series-commander-masters,48.148148
9,Forgotten+Realms+Commander,art-series-commander-masters,48.148148


In [60]:
similarity_df.tail(33)

,cards_set_name,stocks_set_name,Highest Similarity
40,Kaldheim,kaldheim,87.500000
41,Kaldheim,kaldheim,87.500000
42,Magic+2014,magic-2014-m14,66.666667
43,Magic+2014,magic-2014-m14,66.666667
44,Magic+2014,magic-2014-m14,66.666667
45,Magic+2014,magic-2014-m14,66.666667
46,Core+Set+2020,core-set-2020,69.230769
47,Core+Set+2020,core-set-2020,69.230769
48,Core+Set+2020,core-set-2020,69.230769
49,Core+Set+2020,core-set-2020,69.230769


In [64]:
set_rename = {'Forgotten+Realms+Commander': 'Commander: Adventures in the Forgotten Realms', 'Core+Set+2020+Promos': 'Promo Pack: Core Set 2020'}
mtg_cards_df['Set_name'] = mtg_cards_df['Set_name'].replace(set_rename)
mtg_cards_df


,Quantity,Set_name,Name,Rarity,Collector_number.,Foil
0,1,Double+Masters+2022,Darksteel Plate,Rare,304,0.0
1,1,Double+Masters+2022,Mana Drain,Mythic,57,0.0
2,1,Double+Masters+2022,Mana Drain,Mythic,57,1.0
3,1,Commander: Adventures in the Forgotten Realms,Gratuitous Violence,Rare,127,0.0
4,1,Commander: Adventures in the Forgotten Realms,Hurl Through Hell,Rare,48,0.0
...,...,...,...,...,...,...
68,1,Wilds+of+Eldraine+Commander,Court of Ardenvale,Rare,29,0.0
69,1,Wilds+of+Eldraine:+Enchanting+Tales,Smothering Tithe,Mythic,67,0.0
70,2,Zendikar+Rising,Angel of Destiny,Mythic,2,0.0
71,2,Zendikar+Rising,Valakut Awakening // Valakut Stoneforge,Rare,174,0.0


In [70]:
similarities = []

for set_name in mtg_cards_df['Set_name']:
    max_similarity = 0
    
    for set_name_stocks in set_links_df['set_name']:
        similarity = fuzz.ratio(set_name, set_name_stocks)
        if similarity > max_similarity:
            max_similarity = similarity
            best_set_name_stocks = set_name_stocks
        
    similarities.append((set_name, best_set_name_stocks, max_similarity))

Looking good. Now to change mtg_cards_df set_name to set_links_df set name so that the spider can search use the right link when searching for set.

In [66]:
similarity_df = pd.DataFrame(similarities, columns=['cards_set_name', 'stocks_set_name', 'Highest Similarity'])
similarity_df.head(40)

,cards_set_name,stocks_set_name,Highest Similarity
0,Double+Masters+2022,double-masters-2022,78.947368
1,Double+Masters+2022,double-masters-2022,78.947368
2,Double+Masters+2022,double-masters-2022,78.947368
3,Commander: Adventures in the Forgotten Realms,commander-adventures-in-the-forgotten-realms,78.651685
4,Commander: Adventures in the Forgotten Realms,commander-adventures-in-the-forgotten-realms,78.651685
5,Commander: Adventures in the Forgotten Realms,commander-adventures-in-the-forgotten-realms,78.651685
6,Commander: Adventures in the Forgotten Realms,commander-adventures-in-the-forgotten-realms,78.651685
7,Commander: Adventures in the Forgotten Realms,commander-adventures-in-the-forgotten-realms,78.651685
8,Commander: Adventures in the Forgotten Realms,commander-adventures-in-the-forgotten-realms,78.651685
9,Commander: Adventures in the Forgotten Realms,commander-adventures-in-the-forgotten-realms,78.651685


In [67]:
similarity_df.tail(33)

,cards_set_name,stocks_set_name,Highest Similarity
40,Kaldheim,kaldheim,87.500000
41,Kaldheim,kaldheim,87.500000
42,Magic+2014,magic-2014-m14,66.666667
43,Magic+2014,magic-2014-m14,66.666667
44,Magic+2014,magic-2014-m14,66.666667
45,Magic+2014,magic-2014-m14,66.666667
46,Core+Set+2020,core-set-2020,69.230769
47,Core+Set+2020,core-set-2020,69.230769
48,Core+Set+2020,core-set-2020,69.230769
49,Core+Set+2020,core-set-2020,69.230769
